## MLB


Pulling data from mlb game logs and statcast

In [1]:
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:85% !important; }</style>"))

In [ ]:
import urllib
from datetime import timedelta, date
import os
import pickle
import pandas as pd
import json
import csv
import requests
from bs4 import BeautifulSoup
import time

create function to loop through the different days in the schedule to find all games played.


sportid = 1 is MLB

In [ ]:
def get_url(year, month, day):
        
    return 'http://statsapi.mlb.com/api/v1/schedule?sportId=1&date=' + month + '/' + day + '/' + year

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

Save locally and hard-coded the begin and end dates for each season year

In [ ]:
#Scrape all data from given date range according to wind conditions
year = 2015

if year == 2015:
    start_date = date(2015, 4, 5)
    end_date = date(2015, 10, 6)
elif year == 2016:
    start_date = date(2016, 4, 3)
    end_date = date(2016, 10, 3)
elif year == 2017:
    start_date = date(2017, 4, 2)
    end_date = date(2017, 10, 3)
elif year == 2018:
    start_date = date(2018, 3, 29)
    end_date = date(2018, 10, 2)
elif year == 2019:
    start_date = date(2019, 3, 20)
    end_date = date(2019, 10, 1)
elif year == 2014:
    start_date = date(2014, 3, 22)
    end_date = date(2014, 9, 29)
else:
    start_date = date(year, 3, 31)
    end_date = date(year, 4, 1)
    
if not os.path.isdir('MLB/' + str(year) + '_json'):
    os.mkdir('MLB/' + str(year) + '_json')

    
for single_date in daterange(start_date, end_date):
    
    d = single_date.strftime("%Y-%m-%d").split('-')
    day_url = get_url(d[0], d[1], d[2])


    print(day_url)
    with urllib.request.urlopen(day_url) as f:
        webpage = f.read().decode('utf-8')
   
    xml_name = 'MLB/' + str(year) + '_json/' + d[0] + d[1] + d[2] + '.json'
    savefile = open(xml_name, 'w')
    savefile.write(webpage)
    savefile.close()

Now loop through the list of games in csv file to pull down the corresponding json files.

In [ ]:
with open('MLB/MLBGameLog.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
    
def game_url(Game):
    return 'http://statsapi.mlb.com/api/v1.1/game/' + str(Game) + '/feed/live'

if not os.path.isdir('MLB/json'):
    os.mkdir('MLB/json')
    
i = 1

for i in range(len(data)):
    selectedtext = data[i]
    
    #print(selectedtext)
    thisgame = game_url(selectedtext[0])
    #print(thisgame)

    with urllib.request.urlopen(thisgame) as f:
        webpage = f.read().decode('utf-8')
    
    xml_name = 'MLB/json/' + selectedtext[0] + '.json'
    
    if not os.path.exists(xml_name):
        savefile = open(xml_name, 'w')
        savefile.write(webpage)
        savefile.close()
        print(selectedtext[0] + ' downloaded')
        
    #if os.path.exists(xml_name):
        #print(selectedtext[0] + ' already exists')
    
    time.sleep(1)


A distinct list of player_ids was created from aforementioned game logs from 2015 to 2019.  Similar process to pull down desired player info.

In [ ]:
with open('MLB/Player_Ids.csv', encoding='utf-8-sig', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
    
def player_url(Player):
    return 'http://statsapi.mlb.com/api/v1/people/' + Player

if not os.path.isdir('MLB/Players'):
    os.mkdir('MLB/Players')
    
i = 1

for i in range(len(data)):
    selectedtext = data[i]
    
    print(selectedtext[0])
    thisplayer = player_url(selectedtext[0])
    #print(thisgame)

    with urllib.request.urlopen(thisplayer) as f:
        webpage = f.read().decode('utf-8')
    
    xml_name = 'MLB/Players/' + selectedtext[0] + '.json'
    
    if not os.path.exists(xml_name):
        savefile = open(xml_name, 'w')
        savefile.write(webpage)
        savefile.close()
        print(selectedtext[0] + ' downloaded')
        
    #if os.path.exists(xml_name):
        #print(selectedtext[0] + ' already exists')
    
    time.sleep(1)

statcast is below.  This simplified quite a few steps.  By using their python package, you can easily download game logs data although it has already been cleaned and does not include as many features of the raw files from MLB.   


This may help fill in some gaps that may have been missing for whatever reason from the initial source.

In [ ]:
from pybaseball import statcast
import csv
import os

year = 2019

if year == 2015:
    start_date = '2015-4-5'
    end_date = '2015-10-6'
elif year == 2016:
    start_date = '2016-4-3'
    end_date = '2016-10-3'
elif year == 2017:
    start_date = '2017-4-2'
    end_date = '2017-10-3'
elif year == 2018:
    start_date = '2018-3-29'
    end_date = '2018-10-2'
elif year == 2019:
    start_date = '2019-3-20'
    end_date = '2019-10-1'
elif year == 2014:
    start_date = '2014-3-22'
    end_date = '2014-9-29'

data_2019 = statcast(start_date, end_date)
data_2019.to_csv('MLB\statcast\statcast_2019.txt', sep='|', index=False, header=True)